# Analyzing Startup Fundraising Deals From Crunchbase

Throughout this guided project, we'll practice working with different memory constraints. In this step, we assume we only have 10 megabytes of available memory. While *crunchbase-investments.csv* consumes 10.3 megabytes of disk space, we know that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's multiple string columns).

In [147]:
import pandas as pd
import sqlite3

In [148]:
chunk_iter = pd.read_csv("crunchbase-investments.csv", chunksize = 5000, encoding='ISO-8859-1')

In [149]:
# check for missing values across the chunks
missing = list()
for chunk in chunk_iter:
# append each series object we want to combine into a list
    missing.append(chunk.isna().sum())
# combine them using pd.concat()
combined_missing = pd.concat(missing)
# group values associated with the same index then sum to return the final count
missing_final = combined_missing.groupby(combined_missing.index).sum()
print(missing_final.sort_values())

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64


In [150]:
# each column's memory footprints across the chunks
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
counter = 0
series_memory_fp = pd.Series(dtype='float64')
for chunk in chunk_iter:
    if counter == 0:
        series_memory_fp = chunk.memory_usage(deep=True)
    else:
        series_memory_fp += chunk.memory_usage(deep=True)
    counter += 1

# Drop memory footprint calculation for the index.
series_memory_fp = series_memory_fp.drop('Index')
series_memory_fp.sort_values()

raised_amount_usd          422960
funded_year                422960
investor_category_code     622424
investor_state_code       2476607
investor_country_code     2647292
investor_city             2885083
company_state_code        3106051
company_country_code      3172176
funded_month              3383584
funded_quarter            3383584
investor_region           3396281
funding_round_type        3410707
company_region            3411585
company_category_code     3421104
company_city              3505926
funded_at                 3542185
company_name              3591326
investor_name             3915666
company_permalink         4057788
investor_permalink        4980548
dtype: int64

In [151]:
# The total memory footprint of all the chunks combined
series_memory_fp.sum() / (1024 * 1024)

56.9876070022583

In [152]:
# Which column(s) we can drop because they aren't useful for analysis?
# Drop columns representing URLs or containing too many missing values (>90% missing)
drop_cols = ['investor_permalink', 'company_permalink', 'investor_category_code']
keep_cols = chunk.columns.drop(drop_cols)

In [153]:
print(keep_cols)

Index(['company_name', 'company_category_code', 'company_country_code',
       'company_state_code', 'company_region', 'company_city', 'investor_name',
       'investor_country_code', 'investor_state_code', 'investor_region',
       'investor_city', 'funding_round_type', 'funded_at', 'funded_month',
       'funded_quarter', 'funded_year', 'raised_amount_usd'],
      dtype='object')


# Selecting Data Types

In [154]:
# identify the types for each column
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
col_types = {}
for chunk in chunk_iter:
    for col in chunk:
        if col not in col_types:
            col_types[col] = [str(chunk.dtypes[col])]
        else:
            col_types[col].append(str(chunk.dtypes[col]))
print(col_types)
uniq_col_types = {}
for k,v in col_types.items():
    uniq_col_types[k] = set(col_types[k])
uniq_col_types

{'company_permalink': ['object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object'], 'company_name': ['object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object'], 'company_category_code': ['object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object'], 'company_country_code': ['object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object'], 'company_state_code': ['object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object'], 'company_region': ['object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object'], 'company_city': ['object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object'], 'investor_permalink': ['object', 'object', 'object', 'object', 'object', 

{'company_permalink': {'object'},
 'company_name': {'object'},
 'company_category_code': {'object'},
 'company_country_code': {'object'},
 'company_state_code': {'object'},
 'company_region': {'object'},
 'company_city': {'object'},
 'investor_permalink': {'object'},
 'investor_name': {'object'},
 'investor_category_code': {'float64', 'object'},
 'investor_country_code': {'float64', 'object'},
 'investor_state_code': {'float64', 'object'},
 'investor_region': {'object'},
 'investor_city': {'float64', 'object'},
 'funding_round_type': {'object'},
 'funded_at': {'object'},
 'funded_month': {'object'},
 'funded_quarter': {'object'},
 'funded_year': {'float64', 'int64'},
 'raised_amount_usd': {'float64'}}

In [155]:
chunk

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50000,/company/nuorder,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,/person/mortimer-singer,Mortimer Singer,NaN,NaN,NaN,unknown,NaN,series-a,2012-10-01,2012-10,2012-Q4,2012,3060000.0
50001,/company/chacha,ChaCha,advertising,USA,IN,Indianapolis,Carmel,/person/morton-meyerson,Morton Meyerson,NaN,NaN,NaN,unknown,NaN,series-b,2007-10-01,2007-10,2007-Q4,2007,12000000.0
50002,/company/binfire,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,/person/moshe-ariel,Moshe Ariel,NaN,NaN,NaN,unknown,NaN,angel,2008-04-18,2008-04,2008-Q2,2008,500000.0
50003,/company/binfire,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,/person/moshe-ariel,Moshe Ariel,NaN,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,750000.0
50004,/company/unified-color,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,/person/mr-andrew-oung,Mr. Andrew Oung,NaN,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52865,/company/garantia-data,Garantia Data,enterprise,USA,CA,SF Bay,Santa Clara,/person/zohar-gilon,Zohar Gilon,NaN,NaN,NaN,unknown,NaN,series-a,2012-08-08,2012-08,2012-Q3,2012,3800000.0
52866,/company/duda-mobile,DudaMobile,mobile,USA,CA,SF Bay,Palo Alto,/person/zohar-gilon,Zohar Gilon,NaN,NaN,NaN,unknown,NaN,series-c+,2013-04-08,2013-04,2013-Q2,2013,10300000.0
52867,/company/sitebrains,SiteBrains,software,USA,CA,SF Bay,San Francisco,/person/zohar-israel,zohar israel,NaN,NaN,NaN,unknown,NaN,angel,2010-08-01,2010-08,2010-Q3,2010,350000.0
52868,/company/comprehend-systems,Comprehend Systems,enterprise,USA,CA,SF Bay,Palo Alto,/person/zorba-lieberman,Zorba Lieberman,NaN,NaN,NaN,unknown,NaN,series-a,2013-07-11,2013-07,2013-Q3,2013,8400000.0


Identify the numeric columns we can represent using more space efficient types.
For text columns:
1. Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
2. Determine if we can clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
3. Make changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.

# Loading Chunks into SQLite

In [156]:
import sqlite3
conn = sqlite3.connect('crunchbase.db')
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

for chunk in chunk_iter:
    chunk.to_sql("investments", conn, if_exists='append', index=False)

In [163]:
conn = sqlite3.connect('crunchbase.db')
sample=pd.read_sql("SELECT * FROM investments;",conn)
types = pd.read_sql("PRAGMA table_info (investments);",conn)
print(types)

    cid                    name     type  notnull dflt_value  pk
0     0       company_permalink     TEXT        0       None   0
1     1            company_name     TEXT        0       None   0
2     2   company_category_code     TEXT        0       None   0
3     3    company_country_code     TEXT        0       None   0
4     4      company_state_code     TEXT        0       None   0
5     5          company_region     TEXT        0       None   0
6     6            company_city     TEXT        0       None   0
7     7      investor_permalink     TEXT        0       None   0
8     8           investor_name     TEXT        0       None   0
9     9  investor_category_code     TEXT        0       None   0
10   10   investor_country_code     TEXT        0       None   0
11   11     investor_state_code     TEXT        0       None   0
12   12         investor_region     TEXT        0       None   0
13   13           investor_city     TEXT        0       None   0
14   14      funding_roun

# TODO
Use the pandas SQLite workflow to answer the following questions:

1. What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
2. Which category of company attracted the most investments?
3. Which investor contributed the most money (across all startups)?
4. Which investors contributed the most money per startup?
5. Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

Repeat the tasks in this guided project using stricter memory constraints (under 1 megabyte).
Clean and analyze the other Crunchbase datasets from the same GitHub repo.
Understand which columns the datasets share, and how the datasets are linked.
Create a relational database design that links the datasets together and reduces the overall disk space the database file consumes.
Use pandas to populate each table in the database, create the appropriate indexes, and so on.